In [1]:
import json
 
import pandas as pd
from tqdm import tqdm

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/verified_conversations.jsonl'
graph_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/verified_graphs.jsonl'    
conv = pd.read_json(conv_path, lines=True)
graph = pd.read_json(graph_path, lines=True)

In [3]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/conversations_without_graph.jsonl'

In [4]:
sample_index = [782, 142, 631, 1223, 769, 847, 1943, 1322, 1021, 1794, 1749, 188, 1192, 1414, 585, 823, 340, 453, 949, 751, 390, 649, 763, 639, 280, 468, 421, 1743, 1175, 836, 1245, 122, 739, 1452, 50, 787, 686, 1438, 1317, 750, 3, 76, 1648, 876, 403, 1564, 1839, 14, 210, 12, 8, 18, 674, 1721, 1496, 90, 1681]

In [5]:
conv.drop(sample_index, inplace=True)
graph.drop(sample_index, inplace=True)
    


In [6]:
conv = conv.reset_index(drop=True)
graph = graph.reset_index(drop=True)

In [7]:
len(conv), len(graph)

(1932, 1932)

In [ ]:
# for i in range(len(graph)):
#     for j in range(len(graph.iloc[i]['nodes'])):
#         if "port" in graph.iloc[i]['nodes'][j]['type']:
#             graph.iloc[i]['nodes'][j]['type'] = graph.iloc[i]['nodes'][j]['type'].replace(" port", "")
#         graph.iloc[i]['nodes'][j] = graph.iloc[i]['nodes'][j]['content'] + " " + graph.iloc[i]['nodes'][j]['type']

In [8]:
graph.iloc[100]['nodes']

[{'id': 0, 'content': 'clk', 'type': 'input port'},
 {'id': 1, 'content': 'reset', 'type': 'input port'},
 {'id': 2, 'content': 'write_en', 'type': 'input port'},
 {'id': 3, 'content': 'read_reg1_sel', 'type': 'input port'},
 {'id': 4, 'content': 'read_reg2_sel', 'type': 'input port'},
 {'id': 5, 'content': 'write_reg_sel', 'type': 'input port'},
 {'id': 6, 'content': 'write_data', 'type': 'input port'},
 {'id': 7, 'content': 'read_data1', 'type': 'output port'},
 {'id': 8, 'content': 'read_data2', 'type': 'output port'},
 {'id': 9, 'content': 'u_register_array', 'type': 'submodule'},
 {'id': 10, 'content': 'u_control_unit', 'type': 'submodule'}]

In [9]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

In [10]:
system_prompt = 'Please act as a professional Verilog designer. USER:\n'
for i in tqdm(range(len(conv))):
    new_conversations = {}
    # new_graph = "nodes: {}, \nconnectivity: {}".format(graph['nodes'][i], graph['connectivity'][i])
    # text_graph = "\n{}\n".format(new_graph)
    new_instructions = system_prompt + conv.iloc[i]['Instruction']
    module_head = find_module_string(conv.iloc[i]['Response'])
    new_instructions = new_instructions + "\n" + module_head + " ASSISTANT: "
    new_conversations['Instruction'] = new_instructions
    new_conversations['Response'] = conv.iloc[i]['Response'].replace(module_head, "")
    new_conversations['task_id'] = conv.iloc[i]['task_id']
    with open(new_conv_path, 'a') as f:
        json.dump(new_conversations, f)
        f.write('\n')



  0%|          | 0/1932 [00:00<?, ?it/s]

100%|██████████| 1932/1932 [00:00<00:00, 2632.01it/s]


In [11]:
new_conv = pd.read_json(new_conv_path, lines=True)

In [12]:
print(new_conv.iloc[100]['Instruction'])
print("======================")
print(new_conv.iloc[100]['Response'])

Please act as a professional Verilog designer. USER:
Design a 32-bit register bank with read and write capabilities, facilitating parallel read and write operations. The register bank includes eight 32-bit registers that can be accessed individually based on control signals.

Module name:
    register_bank

Function:
A register bank consisting of eight 32-bit registers that supports independent read and write operations simultaneously.

Input ports:
    - clk: Clock signal (1-bit), synchronizes register operations.
    - reset: Reset signal (1-bit), initializes all registers to zero.
    - write_en: Write enable signal (1-bit), controls the writing to a register.
    - read_reg1_sel [2:0]: Selector (3-bits) for choosing which register to read for output1.
    - read_reg2_sel [2:0]: Selector (3-bits) for choosing which register to read for output2.
    - write_reg_sel [2:0]: Selector (3-bits) for choosing which register to write.
    - write_data [31:0]: Data (32-bit) to be written into